In [ ]:
# Uninstall any existing torch, torchvision, and torchaudio packages to avoid conflicts
!pip uninstall -y torch torchvision torchaudio

# Install PyTorch, torchvision, and torchaudio for CUDA 11.8 (A100 GPU)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install the Hugging Face transformers and accelerate libraries
!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate

# Install ipywidgets (optional but useful for widget support in notebooks)
!pip install ipywidgets

Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121
Found existing installation: torchvision 0.19.0+cu121
Uninstalling torchvision-0.19.0+cu121:
  Successfully uninstalled torchvision-0.19.0+cu121
Found existing installation: torchaudio 2.4.0+cu121
Uninstalling torchaudio-2.4.0+cu121:
  Successfully uninstalled torchaudio-2.4.0+cu121
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.6/857.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [ ]:

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from google.colab import drive
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import torch

# Mount Google Drive to save the output
drive.mount('/content/drive')

# Load the CSV file (update the path to your CSV)
csv_path = '/content/test.csv'  # Adjust to your path
df = pd.read_csv(csv_path)

# Initialize the Hugging Face model, tokenizer, and processor
# model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen-2VL").to("cuda")
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-2VL")
# processor = AutoProcessor.from_pretrained("Qwen/Qwen-2VL")

# Function to download image from URL
def download_image(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((512, 512))
        return img
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None

# Process the data in batches of 1000 rows
batch_size = 1000
for start_idx in range(36000, len(df), batch_size):
    end_idx = min(start_idx + batch_size, len(df))
    batch_df = df.iloc[start_idx:end_idx]

    print(f"Processing batch {start_idx} to {end_idx}")

    extracted_data = []

    for index, row in batch_df.iterrows():
        image_link = row['image_link']
        entity_name = row['entity_name']
        group_id = row['group_id']

        # Calculate the actual row number relative to the full dataframe
        global_row_number = start_idx + index + 1  # Adding 1 to make it 1-based index

        print(f"Processing row {global_row_number} out of {len(df)}")

        # Download the image
        image = download_image(image_link)

        if image is not None:
            # Create the message for the model
            messages = [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                        },
                        {
                            "type": "text",
                            "text": f"Output only {entity_name} value and units only from the text in the image."
                        }
                    ]
                }
            ]

            # Prepare text prompt and inputs
            text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
            inputs = processor(text=[text_prompt], images=[image], padding=True, return_tensors="pt").to("cuda")

            # Generate output
            with torch.no_grad():
                output_ids = model.generate(**inputs, max_new_tokens=256)

            # Extract and decode the output
            generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
            output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

            extracted_text = output_text[0] if output_text else ""
            print(f"Batch {start_idx} to {end_idx} - Processing Image: {image_link}")
            print(f"Extracted Data: {extracted_text}")

            # Append the result
            extracted_data.append({
                "image_link": image_link,
                "entity_name": entity_name,
                "group_id": group_id,
                "extracted_answer": extracted_text
            })
        else:
            print(f"Image download failed for {image_link}")
            extracted_data.append({
                "image_link": image_link,
                "entity_name": entity_name,
                "group_id": group_id,
                "extracted_answer": "Image download failed"
            })

    # Convert extracted data to DataFrame
    result_df = pd.DataFrame(extracted_data)

    # Save the batch to Google Drive
    output_csv_path = f'/content/drive/My Drive/new_extracted_results_batch_{start_idx}_{end_idx}.csv'
    result_df.to_csv(output_csv_path, index=False)
    print(f"Batch {start_idx} to {end_idx} saved to {output_csv_path}\n")


Streaming output truncated to the last 5000 lines.
Batch 68000 to 69000 - Processing Image: https://m.media-amazon.com/images/I/51qOQjkSe3L.jpg
Extracted Data: 18cm
Processing row 136576 out of 131187
Batch 68000 to 69000 - Processing Image: https://m.media-amazon.com/images/I/51qOQjkSe3L.jpg
Extracted Data: 18cm
Processing row 136577 out of 131187
Batch 68000 to 69000 - Processing Image: https://m.media-amazon.com/images/I/51qOQjkSe3L.jpg
Extracted Data: 18cm
Processing row 136578 out of 131187
Batch 68000 to 69000 - Processing Image: https://m.media-amazon.com/images/I/51qOSFPLUiL.jpg
Extracted Data: The fan in the image is a Tianxuan TX9225M12S, which is a 92mm x 92mm x 25mm (3.63 inches x 3.63 inches x 0.98 inches) DC 12V 0.19A server fan.
Processing row 136579 out of 131187
Batch 68000 to 69000 - Processing Image: https://m.media-amazon.com/images/I/51qOSFPLUiL.jpg
Extracted Data: The fan is a Tianxuan TX9225M12S, which is a 92mm x 92mm x 25mm (3.63 inches x 3.63 inches x 0.98 inc

KeyboardInterrupt: 